In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 10
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    
    hiddenSizes = [128]  # LSTM结构的神经元个数
    
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
    outputSize = 128  # 从高维映射到低维的神经元个数
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000, 1)
eval data shape: (5000, 200)


In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [6]:
"""
构建模型，模型的架构如下：
1，利用Bi-LSTM获得上下文的信息
2，将Bi-LSTM获得的隐层输出和词向量拼接[fwOutput;wordEmbedding;bwOutput]
3，将2所得的词表示映射到低维
4，hidden_size上每个位置的值都取时间步上最大的值，类似于max-pool
5，softmax分类
"""

class RCNN(object):
    """
    RCNN 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            # 复制一份embedding input
            self.embeddedWords_ = self.embeddedWords
            
        # 定义两层双向LSTM的模型结构

#         with tf.name_scope("Bi-LSTM"):
#             fwHiddenLayers = []
#             bwHiddenLayers = []
#             for idx, hiddenSize in enumerate(config.model.hiddenSizes):

#                 with tf.name_scope("Bi-LSTM-" + str(idx)):
#                     # 定义前向LSTM结构
#                     lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
#                                                                  output_keep_prob=self.dropoutKeepProb)
#                     # 定义反向LSTM结构
#                     lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
#                                                                  output_keep_prob=self.dropoutKeepProb)

#                 fwHiddenLayers.append(lstmFwCell)
#                 bwHiddenLayers.append(lstmBwCell)

#             # 实现多层的LSTM结构， state_is_tuple=True，则状态会以元祖的形式组合(h, c)，否则列向拼接
#             fwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=fwHiddenLayers, state_is_tuple=True)
#             bwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=bwHiddenLayers, state_is_tuple=True)

#             # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
#             # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
#             # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
#             outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(fwMultiLstm, bwMultiLstm, self.embeddedWords, dtype=tf.float32)
#             fwOutput, bwOutput = outputs

        with tf.name_scope("Bi-LSTM"):
            for idx, hiddenSize in enumerate(config.model.hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向LSTM结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)
                    # 定义反向LSTM结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                                                                 output_keep_prob=self.dropoutKeepProb)


                    # 采用动态rnn，可以动态的输入序列的长度，若没有输入，则取序列的全长
                    # outputs是一个元祖(output_fw, output_bw)，其中两个元素的维度都是[batch_size, max_time, hidden_size],fw和bw的hidden_size一样
                    # self.current_state 是最终的状态，二元组(state_fw, state_bw)，state_fw=[batch_size, s]，s是一个元祖(h, c)
                    outputs_, self.current_state = tf.nn.bidirectional_dynamic_rnn(lstmFwCell, lstmBwCell, 
                                                                                  self.embeddedWords_, dtype=tf.float32,
                                                                                  scope="bi-lstm" + str(idx))
        
                    # 对outputs中的fw和bw的结果拼接 [batch_size, time_step, hidden_size * 2], 传入到下一层Bi-LSTM中
                    self.embeddedWords_ = tf.concat(outputs_, 2)
                
        # 将最后一层Bi-LSTM输出的结果分割成前向和后向的输出
        fwOutput, bwOutput = tf.split(self.embeddedWords_, 2, -1)
            
        with tf.name_scope("context"):
            shape = [tf.shape(fwOutput)[0], 1, tf.shape(fwOutput)[2]]
            self.contextLeft = tf.concat([tf.zeros(shape), fwOutput[:, :-1]], axis=1, name="contextLeft")
            self.contextRight = tf.concat([bwOutput[:, 1:], tf.zeros(shape)], axis=1, name="contextRight")
            
        # 将前向，后向的输出和最早的词向量拼接在一起得到最终的词表征
        with tf.name_scope("wordRepresentation"):
            self.wordRepre = tf.concat([self.contextLeft, self.embeddedWords, self.contextRight], axis=2)
            wordSize = config.model.hiddenSizes[-1] * 2 + config.model.embeddingSize 
        
        with tf.name_scope("textRepresentation"):
            outputSize = config.model.outputSize
            textW = tf.Variable(tf.random_uniform([wordSize, outputSize], -1.0, 1.0), name="W2")
            textB = tf.Variable(tf.constant(0.1, shape=[outputSize]), name="b2")
            
            # tf.einsum可以指定维度的消除运算
            self.textRepre = tf.tanh(tf.einsum('aij,jk->aik', self.wordRepre, textW) + textB)
            
        # 做max-pool的操作，将时间步的维度消失
        output = tf.reduce_max(self.textRepre, axis=1)
        
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss

In [7]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [8]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        lstm = RCNN(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(lstm.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", lstm.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
#         builder = tf.saved_model.builder.SavedModelBuilder("../model/Bi-LSTM/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              lstm.inputX: batchX,
              lstm.inputY: batchY,
              lstm.dropoutKeepProb: 1.0
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, lstm.loss, lstm.predictions, lstm.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
#                 if currentStep % config.training.checkpointEvery == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/Bi-LSTM/model/my-model", global_step=currentStep)
#                     print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(lstm.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(lstm.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(lstm.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

Tensor("context/contextLeft:0", shape=(?, 200, 128), dtype=float32)
Tensor("context/contextRight:0", shape=(?, 200, 128), dtype=float32)
Tensor("wordRepresentation/concat:0", shape=(?, 200, 456), dtype=float32)
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/kernel:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/hist is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name bi-lstm0/fw/lstm_cell/bias:0/grad/sparsity is illegal; using bi-lstm0/fw/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorfl

2019-03-06T16:31:49.153232, step: 50, loss: 0.6995680928230286, acc: 0.5, auc: 0.5171, precision: 0.0, recall: 0.0
2019-03-06T16:31:49.534184, step: 51, loss: 0.6941133737564087, acc: 0.4531, auc: 0.4042, precision: 0.0, recall: 0.0
2019-03-06T16:31:49.915198, step: 52, loss: 0.7046807408332825, acc: 0.5312, auc: 0.5586, precision: 0.0, recall: 0.0
2019-03-06T16:31:50.286173, step: 53, loss: 0.7045686841011047, acc: 0.5391, auc: 0.4949, precision: 1.0, recall: 0.0167
2019-03-06T16:31:50.676238, step: 54, loss: 0.6949774026870728, acc: 0.4922, auc: 0.4857, precision: 0.0, recall: 0.0
2019-03-06T16:31:51.057252, step: 55, loss: 0.697566032409668, acc: 0.5234, auc: 0.5143, precision: 0.0, recall: 0.0
2019-03-06T16:31:51.443344, step: 56, loss: 0.6845508813858032, acc: 0.4297, auc: 0.6017, precision: 0.0, recall: 0.0
2019-03-06T16:31:51.821303, step: 57, loss: 0.6892661452293396, acc: 0.4609, auc: 0.5652, precision: 0.0, recall: 0.0
2019-03-06T16:31:52.205276, step: 58, loss: 0.69354367256

2019-03-06T16:32:30.690924, step: 118, loss: 0.6970404386520386, acc: 0.5078, auc: 0.5414, precision: 0.0, recall: 0.0
2019-03-06T16:32:31.086834, step: 119, loss: 0.682064414024353, acc: 0.5391, auc: 0.5905, precision: 0.0, recall: 0.0
2019-03-06T16:32:31.475825, step: 120, loss: 0.6814188957214355, acc: 0.4844, auc: 0.6227, precision: 0.0, recall: 0.0
2019-03-06T16:32:31.870769, step: 121, loss: 0.6842150688171387, acc: 0.5547, auc: 0.592, precision: 0.0, recall: 0.0
2019-03-06T16:32:32.265683, step: 122, loss: 0.6854759454727173, acc: 0.4766, auc: 0.5696, precision: 0.0, recall: 0.0
2019-03-06T16:32:32.658835, step: 123, loss: 0.6752969622612, acc: 0.4297, auc: 0.6169, precision: 0.0, recall: 0.0
2019-03-06T16:32:33.040816, step: 124, loss: 0.6952329874038696, acc: 0.5859, auc: 0.5907, precision: 0.0, recall: 0.0
2019-03-06T16:32:33.449867, step: 125, loss: 0.6755962371826172, acc: 0.4922, auc: 0.6474, precision: 0.0, recall: 0.0
2019-03-06T16:32:33.826860, step: 126, loss: 0.673463

2019-03-06T16:32:56.871191, step: 185, loss: 0.47419634461402893, acc: 0.8203, auc: 0.8789, precision: 0.85, recall: 0.7846
2019-03-06T16:32:57.265278, step: 186, loss: 0.45825594663619995, acc: 0.8281, auc: 0.9064, precision: 0.8136, recall: 0.8136
2019-03-06T16:32:57.640295, step: 187, loss: 0.509143054485321, acc: 0.7734, auc: 0.8389, precision: 0.7941, recall: 0.7826
2019-03-06T16:32:58.020258, step: 188, loss: 0.5246121287345886, acc: 0.7422, auc: 0.8365, precision: 0.7458, recall: 0.7097
2019-03-06T16:32:58.409217, step: 189, loss: 0.45105502009391785, acc: 0.8516, auc: 0.8614, precision: 0.8772, recall: 0.8065
2019-03-06T16:32:58.787207, step: 190, loss: 0.4695312976837158, acc: 0.7734, auc: 0.8604, precision: 0.871, recall: 0.72
2019-03-06T16:32:59.162236, step: 191, loss: 0.48737162351608276, acc: 0.7891, auc: 0.8532, precision: 0.8462, recall: 0.7639
2019-03-06T16:32:59.560257, step: 192, loss: 0.46560007333755493, acc: 0.8438, auc: 0.8851, precision: 0.8269, recall: 0.7963
2

2019-03-06T16:33:37.912479, step: 250, loss: 0.38814878463745117, acc: 0.8438, auc: 0.9104, precision: 0.8475, recall: 0.8197
2019-03-06T16:33:38.291435, step: 251, loss: 0.4007378816604614, acc: 0.8281, auc: 0.8997, precision: 0.9375, recall: 0.7031
2019-03-06T16:33:38.664540, step: 252, loss: 0.3800564706325531, acc: 0.8203, auc: 0.9163, precision: 0.9184, recall: 0.7031
2019-03-06T16:33:39.044549, step: 253, loss: 0.3425327241420746, acc: 0.8281, auc: 0.9336, precision: 0.8966, recall: 0.7647
2019-03-06T16:33:39.424659, step: 254, loss: 0.3951859772205353, acc: 0.8047, auc: 0.9076, precision: 0.8475, recall: 0.7576
2019-03-06T16:33:39.806668, step: 255, loss: 0.36047887802124023, acc: 0.875, auc: 0.9316, precision: 0.8871, recall: 0.8594
2019-03-06T16:33:40.182632, step: 256, loss: 0.39444535970687866, acc: 0.8594, auc: 0.9224, precision: 0.7971, recall: 0.9322
2019-03-06T16:33:40.564611, step: 257, loss: 0.3307536244392395, acc: 0.8984, auc: 0.9375, precision: 0.8793, recall: 0.894

...

2019-03-06T16:42:28.311313, step: 1233, loss: 0.025818243622779846, acc: 0.9922, auc: 0.9998, precision: 0.9836, recall: 1.0
2019-03-06T16:42:28.695286, step: 1234, loss: 0.11006683111190796, acc: 0.9688, auc: 0.9775, precision: 0.9545, recall: 0.9844
2019-03-06T16:42:29.085243, step: 1235, loss: 0.05900188535451889, acc: 0.9844, auc: 0.9938, precision: 1.0, recall: 0.9643
2019-03-06T16:42:29.489383, step: 1236, loss: 0.03538547456264496, acc: 0.9844, auc: 0.9998, precision: 0.9841, recall: 0.9841
2019-03-06T16:42:29.870334, step: 1237, loss: 0.06960602849721909, acc: 0.9844, auc: 0.984, precision: 0.9865, recall: 0.9865
2019-03-06T16:42:30.246479, step: 1238, loss: 0.06697320938110352, acc: 0.9844, auc: 0.9898, precision: 0.9818, recall: 0.9818
2019-03-06T16:42:30.640425, step: 1239, loss: 0.03037678636610508, acc: 0.9922, auc: 1.0, precision: 0.9836, recall: 1.0
2019-03-06T16:42:31.048335, step: 1240, loss: 0.0516471341252327, acc: 0.9922, auc: 0.999, precision: 0.9846, recall: 1.0
2

2019-03-06T16:43:10.532923, step: 1300, loss: 0.5888405151856251, acc: 0.8585794871794872, auc: 0.9153717948717949, precision: 0.8605897435897436, recall: 0.8601923076923075
2019-03-06T16:43:10.911912, step: 1301, loss: 0.1340472400188446, acc: 0.9766, auc: 0.9792, precision: 0.9683, recall: 0.9839
2019-03-06T16:43:11.289901, step: 1302, loss: 0.048586077988147736, acc: 0.9844, auc: 0.9971, precision: 1.0, recall: 0.9667
2019-03-06T16:43:11.671997, step: 1303, loss: 0.05303336679935455, acc: 0.9844, auc: 0.9961, precision: 1.0, recall: 0.9688
2019-03-06T16:43:12.062921, step: 1304, loss: 0.006234294269233942, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-03-06T16:43:12.440094, step: 1305, loss: 0.07055431604385376, acc: 0.9766, auc: 0.9988, precision: 0.9846, recall: 0.9697
2019-03-06T16:43:12.812067, step: 1306, loss: 0.050517499446868896, acc: 0.9844, auc: 0.9995, precision: 1.0, recall: 0.9706
2019-03-06T16:43:13.182109, step: 1307, loss: 0.00920890737324953, acc: 1.0, auc: 1

2019-03-06T16:43:37.323354, step: 1368, loss: 0.04650575667619705, acc: 0.9922, auc: 0.9975, precision: 1.0, recall: 0.9804
2019-03-06T16:43:37.716273, step: 1369, loss: 0.010333584621548653, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-03-06T16:43:38.122213, step: 1370, loss: 0.04885723069310188, acc: 0.9922, auc: 0.9928, precision: 1.0, recall: 0.9861
2019-03-06T16:43:38.516257, step: 1371, loss: 0.04750198870897293, acc: 0.9922, auc: 0.997, precision: 0.9828, recall: 1.0
2019-03-06T16:43:38.903251, step: 1372, loss: 0.0604175440967083, acc: 0.9844, auc: 0.9956, precision: 0.9851, recall: 0.9851
2019-03-06T16:43:39.322234, step: 1373, loss: 0.005827221553772688, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-03-06T16:43:39.720198, step: 1374, loss: 0.09743715822696686, acc: 0.9766, auc: 0.9863, precision: 0.971, recall: 0.9853
2019-03-06T16:43:40.142043, step: 1375, loss: 0.09328581392765045, acc: 0.9844, auc: 0.9897, precision: 0.9815, recall: 0.9815
2019-03-06T16:43:4

2019-03-06T16:44:17.965667, step: 1434, loss: 0.07743426412343979, acc: 0.9844, auc: 0.9927, precision: 0.9848, recall: 0.9848
2019-03-06T16:44:18.361703, step: 1435, loss: 0.010762064717710018, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-03-06T16:44:18.755653, step: 1436, loss: 0.05526698753237724, acc: 0.9922, auc: 0.9907, precision: 0.9853, recall: 1.0
2019-03-06T16:44:19.139624, step: 1437, loss: 0.010822732001543045, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-03-06T16:44:19.511629, step: 1438, loss: 0.0066413888707757, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-03-06T16:44:19.900557, step: 1439, loss: 0.006833563558757305, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-03-06T16:44:20.305475, step: 1440, loss: 0.048160385340452194, acc: 0.9922, auc: 0.9939, precision: 0.9857, recall: 1.0
2019-03-06T16:44:20.708555, step: 1441, loss: 0.005552802234888077, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-03-06T16:44:21.111508, step: 1442, loss: 

2019-03-06T16:45:00.449990, step: 1501, loss: 0.011677064001560211, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-03-06T16:45:00.860864, step: 1502, loss: 0.01839117705821991, acc: 0.9922, auc: 1.0, precision: 1.0, recall: 0.9857
2019-03-06T16:45:01.233868, step: 1503, loss: 0.07604608684778214, acc: 0.9766, auc: 0.9985, precision: 1.0, recall: 0.9516
2019-03-06T16:45:01.666709, step: 1504, loss: 0.04400978982448578, acc: 0.9922, auc: 0.9995, precision: 1.0, recall: 0.9851
2019-03-06T16:45:02.038745, step: 1505, loss: 0.00975782424211502, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-03-06T16:45:02.417731, step: 1506, loss: 0.01306744571775198, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-03-06T16:45:02.794826, step: 1507, loss: 0.02674892172217369, acc: 0.9922, auc: 0.9998, precision: 0.9865, recall: 1.0
2019-03-06T16:45:03.174953, step: 1508, loss: 0.10819491744041443, acc: 0.9844, auc: 0.9743, precision: 0.9677, recall: 1.0
2019-03-06T16:45:03.564939, step: 150